In [38]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import pickle

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

## Step 1: Read in hold out data, scalers, and best model

In [39]:
# final_scaler = read_pickle(filename)
# final_model = read_pickle(filename)

In [40]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)
holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [41]:
pickle_in = open("model.pickle", "rb")
model = pickle.load(pickle_in)

In [42]:
len(model.coef_)

100

In [43]:
selected_columns = pd.read_csv('selected_columns_bestmodel.csv')

In [44]:
selected_columns

,Unnamed: 0,0
0,0,bedrooms
1,1,bathrooms
2,2,sqft_living
3,3,floors
4,4,view
...,...,...
95,95,lat waterfront_1
96,96,long waterfront_1
97,97,age waterfront_1
98,98,condition_3 waterfront_1


In [45]:
selected_columns.drop('Unnamed: 0', axis=1, inplace=True)

In [46]:
selected_columns

,0
0,bedrooms
1,bathrooms
2,sqft_living
3,floors
4,view
...,...
95,lat waterfront_1
96,long waterfront_1
97,age waterfront_1
98,condition_3 waterfront_1


In [47]:
select_col = list(selected_columns['0'].values)
# select_col

In [48]:
len(select_col)

100

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [49]:
holdout

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,263000018,20140521T000000,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
4319,6600060120,20150223T000000,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
4320,1523300141,20140623T000000,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
4321,291310100,20150116T000000,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [50]:
holdout.drop(columns=['date'], inplace = True)

In [51]:
holdout['bedrooms'] = holdout['bedrooms'].replace([0,33,11],[holdout['bedrooms'].median(),3, 4])
holdout['bedrooms']

0       4.0
1       4.0
2       3.0
3       3.0
4       4.0
       ... 
4318    3.0
4319    4.0
4320    2.0
4321    3.0
4322    2.0
Name: bedrooms, Length: 4323, dtype: float64

In [52]:
holdout['age'] = 2020 - holdout['yr_built']

In [53]:
holdout['renovation_age'] = np.where(holdout['yr_renovated']>0, 2020 - holdout['yr_renovated'], 0)

In [54]:
no_basement = holdout[holdout['sqft_basement'] == 0]
basement = holdout[holdout['sqft_basement'] != 0]

In [55]:
waterfront = holdout[holdout['waterfront'] == 1]
no_water = holdout[holdout['waterfront'] == 0]

In [56]:
df_dummy = pd.get_dummies(holdout, columns=['condition', 'waterfront'], drop_first='True')

In [57]:
df_dummy.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'view', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'age', 'renovation_age', 'condition_2', 'condition_3', 'condition_4',
       'condition_5', 'waterfront_1'],
      dtype='object')

In [58]:
exclude = ['id', 'price', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']

In [59]:
features = [x for x in df_dummy.columns if x not in exclude]

In [33]:
df_dummy.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'view', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'age', 'renovation_age', 'condition_2', 'condition_3', 'condition_4',
       'condition_5', 'waterfront_1'],
      dtype='object')

In [60]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_data = poly.fit_transform(df_dummy[features])
poly_data

array([[4.00e+00, 2.50e+00, 2.27e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [4.00e+00, 2.50e+00, 2.27e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [3.00e+00, 2.50e+00, 1.47e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       ...,
       [2.00e+00, 7.50e-01, 1.02e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [3.00e+00, 2.50e+00, 1.60e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [2.00e+00, 7.50e-01, 1.02e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00]])

In [61]:
poly_data_columns = poly.get_feature_names(features)
holdout_poly = pd.DataFrame(poly_data, columns=poly_data_columns)

In [68]:
# poly_2 = PolynomialFeatures(degree=2, include_bias=False)
# poly2_data = poly_2.fit_transform(df_dummy[features])
# poly2_columns = poly_2.get_feature_names(features)
# hold_out_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
# hold_out_poly2.head()

In [65]:
selected_columns

,0
0,bedrooms
1,bathrooms
2,sqft_living
3,floors
4,view
...,...
95,lat waterfront_1
96,long waterfront_1
97,age waterfront_1
98,condition_3 waterfront_1


In [64]:
holdout_poly[selected_columns]

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,grade,sqft_above,sqft_basement,zipcode,lat,long,age,renovation_age,condition_2,condition_3,condition_4,condition_5,waterfront_1,bedrooms^2,bedrooms bathrooms,bedrooms sqft_living,bedrooms sqft_lot,bedrooms floors,bedrooms view,bedrooms grade,bedrooms sqft_above,bedrooms sqft_basement,bedrooms zipcode,bedrooms lat,bedrooms long,bedrooms age,bedrooms renovation_age,bedrooms condition_2,bedrooms condition_3,bedrooms condition_4,bedrooms condition_5,bedrooms waterfront_1,bathrooms^2,bathrooms sqft_living,bathrooms sqft_lot,bathrooms floors,bathrooms view,bathrooms grade,bathrooms sqft_above,bathrooms sqft_basement,bathrooms zipcode,bathrooms lat,bathrooms long,bathrooms age,...,zipcode condition_2,zipcode condition_3,zipcode condition_4,zipcode condition_5,zipcode waterfront_1,lat^2,lat long,lat age,lat renovation_age,lat condition_2,lat condition_3,lat condition_4,lat condition_5,lat waterfront_1,long^2,long age,long renovation_age,long condition_2,long condition_3,long condition_4,long condition_5,long waterfront_1,age^2,age renovation_age,age condition_2,age condition_3,age condition_4,age condition_5,age waterfront_1,renovation_age^2,renovation_age condition_2,renovation_age condition_3,renovation_age condition_4,renovation_age condition_5,renovation_age waterfront_1,condition_2^2,condition_2 condition_3,condition_2 condition_4,condition_2 condition_5,condition_2 waterfront_1,condition_3^2,condition_3 condition_4,condition_3 condition_5,condition_3 waterfront_1,condition_4^2,condition_4 condition_5,condition_4 waterfront_1,condition_5^2,condition_5 waterfront_1,waterfront_1^2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [69]:
# hold_out_poly2.shape

In [124]:
# scaled_holdout = scaler.transform(hold_out_poly2[select_col])

In [ ]:
# preds = final_model.predict(scaled_holdout)

In [68]:
y_pred = model.predict(holdout_poly[select_col])

In [69]:
y_pred

array([684927.08050728, 684927.08050728, 412147.21319866, ...,
       366347.87766933, 436446.45437241, 367263.41966534])

In [39]:
# poly.get_feature_names(features)

In [37]:
poly_data

array([[2.5000e+00, 4.0000e+00, 9.8034e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.5000e+00, 4.0000e+00, 9.8034e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.5000e+00, 3.0000e+00, 9.8029e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [7.5000e-01, 2.0000e+00, 9.8144e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.5000e+00, 3.0000e+00, 9.8027e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.5000e-01, 2.0000e+00, 9.8144e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [82]:
# transformed_holdout = final_scaler(holdout)

## Step 3: Predict the holdout set

In [38]:
model_holdout = model.transform(hold_out_poly2[select_col])

AttributeError: 'LinearRegression' object has no attribute 'transform'

In [39]:
hold_out_poly2[select_col]

KeyError: "['bedrooms grade_11', 'zipcode grade_7', 'condition_4 grade_8', 'condition_3 grade_11', 'grade_13', 'renovation_age grade_11', 'renovation_age zipcode', 'condition_5 grade_12', 'grade_7', 'condition_4 grade_7', 'age grade_10', 'bathrooms grade_7', 'grade_9^2', 'renovation_age grade_6', 'bedrooms grade_13', 'condition_5 grade_7', 'grade_12^2', 'condition_4 grade_6', 'grade_4^2', 'condition_2 grade_6', 'bedrooms grade_9', 'grade_6^2', 'condition_4 grade_9', 'grade_5', 'condition_5 grade_11', 'bedrooms grade_12', 'condition_5 grade_8', 'grade_6', 'bedrooms grade_5', 'age grade_13', 'bedrooms grade_7', 'renovation_age grade_13', 'age grade_5', 'condition_4 grade_10', 'renovation_age grade_9', 'age grade_6', 'condition_3 grade_8', 'grade_9', 'zipcode grade_10', 'condition_4 grade_13', 'bathrooms grade_5', 'condition_4 grade_5', 'grade_10^2', 'zipcode grade_12', 'age grade_9', 'condition_2 grade_10', 'bathrooms grade_4', 'condition_3 grade_6', 'age grade_7', 'age grade_4', 'grade_

In [83]:
# final_answers = final_model.predict(transformed_holdout)

In [70]:
y_pred = model.predict(poly_data)
y_pred

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 209)

## Step 4: Export your predictions

In [85]:
# final_answer.to_csv('housing_preds_your_name.csv')

In [71]:
pd.DataFrame(y_pred).to_csv('housing_pred_Raizel_Bernstein_2.csv')